In [1]:
from operator import itemgetter

In [104]:
def voting(sc, rdds):
    ensemble_rdd = sc.emptyRDD()
    
    for rdd in rdds:
        ensemble_rdd = ensemble_rdd.union(rdd.map(lambda x: (x[0], [(i,1) for i in x[1]])))
    
    voting_rdd = (ensemble_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                     .map(lambda x: ((x[0],x[1][0]),x[1][1]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                     )
    
    return voting_rdd

def voting_weighted(sc, rdds):
    ensemble_rdd = sc.emptyRDD()
    
    for rdd in rdds:
        value = rdd[1]
        ensemble_rdd = ensemble_rdd.union(rdd[0].map(lambda x: (x[0], [(i,value) for i in x[1]])))
    
    voting_rdd = (ensemble_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                     .map(lambda x: ((x[0],x[1][0]),x[1][1]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                     )
    
    return voting_rdd

In [105]:
def recommendation_list(string):
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i)))
            
    return lista

def parse_submission(sc, path):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list(x[1])))
    )

    return rdd

In [106]:
a = parse_submission(sc, "/Users/tommy/Documents/pumpkin-pie/top_pop_impressions.csv")
b = parse_submission(sc, "/Users/tommy/Documents/pumpkin-pie/top_pop_impressions.csv")

print a.take(1)

[(9, [2778525, 1244196, 1053542, 657183, 536047, 1386412, 1162250, 1583705, 2791339, 784737, 1984327, 343377, 1576126, 2106311, 1056667, 830073, 1953846, 2446769, 1679143, 2031981, 1069281, 468120, 1851386, 1776330, 2593483, 2466095, 1443706, 2002097, 1092821, 460717])]


In [84]:
recommendation_rdd = voting(sc, [a,b])

In [107]:
recommendation_rdd = voting_weighted(sc, [(a,0.5), (b,1)])

In [108]:
recommendation_rdd.take(1)

[(458752,
  [(1443706, 1.5),
   (1953846, 1.5),
   (1576126, 1.5),
   (1053542, 1.5),
   (1162250, 1.5),
   (1851386, 1.5),
   (1776330, 1.5),
   (2031981, 1.5),
   (830073, 1.5),
   (784737, 1.5),
   (1092821, 1.5),
   (460717, 1.5),
   (2002097, 1.5),
   (343377, 1.5),
   (2446769, 1.5),
   (1583705, 1.5),
   (1069281, 1.5),
   (2778525, 1.5),
   (1386412, 1.5),
   (1244196, 1.5),
   (468120, 1.5),
   (2106311, 1.5),
   (2593483, 1.5),
   (2466095, 1.5),
   (536047, 1.5),
   (1984327, 1.5),
   (1679143, 1.5),
   (2791339, 1.5),
   (1056667, 1.5),
   (657183, 1.5)])]